In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
%matplotlib inline

In [ ]:
my_cols=['c1','c2','c3','c4','c5','c6','c7','c8','c9']
with open('../data/sample-laser-radar-measurement-data-1.txt') as f:
    table1 = pd.read_table(f, sep='\t', header=None, names=my_cols, lineterminator='\n')

with open('../data/sample-laser-radar-measurement-data-2.txt') as f:
    table2 = pd.read_table(f, sep='\t', header=None, names=my_cols, lineterminator='\n')
    
table = pd.concat([table1,table2])


laser_df = table[table['c1']=='L']
del laser_df['c9']
del laser_df['c1']
laser_df.columns = ['x','y','timestamp','gpx','gpy','gvx','gvy']

radar_df = table[table['c1']=='R']
del radar_df['c1']
radar_df.columns = ['rho','phi','rho_dot','timestamp','gpx','gpy','gvx','gvy']

In [ ]:
laser_df.head(5)

In [ ]:
radar_df.head(5)

### Extract the x, y data and the corresponding ground truth data

In [ ]:
xdata=np.array(laser_df['x'].values)
gpxdata=np.array(laser_df['gpx'].values)
vxdata=gpxdata-xdata
ydata=np.array(laser_df['y'].values)
gpydata=np.array(laser_df['gpy'].values)
vydata=gpydata-ydata

In [ ]:
plt.plot(xdata, ydata, 'b-', label='data')
plt.plot(gpxdata, gpydata, 'g-', label='gt')

### Have a look at the difference between the x, y and ground truth x y

In [ ]:
plt.plot(vxdata)

In [ ]:
plt.plot(vydata)

### Variance in the differences (laser data)

In [ ]:
np.var(vxdata), np.var(vydata)

### Variance in the differences (radar data) - converted between cartesian/polar

In [ ]:
def calc_hx(gpx,gpy,gvx,gvy):
    if gpx == 0:
        return 0,0,0
    rho = math.sqrt(gpx*gpx + gpy*gpy)
    phi = math.atan(gpy/gpx)
    rho_dot = (gpx*gvx + gpy*gvy)/rho
    return np.array([rho,phi,rho_dot])

In [ ]:
ghs=[ np.hstack([z[0], z[1], z[2], calc_hx(z[3], z[4], z[5], z[6])]) for z in radar_df.as_matrix(['rho', 'phi', 'rho_dot', 'gpx', 'gpy', 'gvx', 'gvy'])]

gr_df = pd.DataFrame(ghs)
gr_df.columns = ['rho', 'phi', 'rho_dot', 'grho', 'gphi', 'grho_dot']
vrhodata=np.array(gr_df['grho'].values -gr_df['rho'].values)
vphidata=np.array(gr_df['gphi'].values -gr_df['phi'].values)
vrhodotdata=np.array(gr_df['grho_dot'].values -gr_df['rho_dot'].values)
#gr_df

In [ ]:
np.var(vrhodata), np.var(vphidata), np.var(vrhodotdata)

In [ ]:
cov_table = table.copy()
cov_table.reindex()
del cov_table['c7']
del cov_table['c8']
del cov_table['c9']
#cov_table.columns = ['px','py','v','gpx','gpy','gvx','gvy']
cov_table.cov()